In [18]:
import pandas as pd
import sklearn
import sklearn.datasets
import sklearn.ensemble
import numpy as np
import sklearn.model_selection
from __future__ import print_function

In [19]:
dataml =  np.genfromtxt('ml1data.train.txt', delimiter=',', dtype=str)

In [20]:
labels = dataml[:,21]
le= sklearn.preprocessing.LabelEncoder()
le.fit(labels)
labels = le.transform(labels)
class_names = le.classes_
dataml = dataml[:,:-1]

In [21]:
dataml

array([['A', '138', 'E', ..., '71.81', 'DP', 'DO'],
       ['A', '966', 'E', ..., '71.81', 'NN', 'DO'],
       ['C', '345', 'E', ..., '71.81', 'DP', 'DO'],
       ..., 
       ['C', '276', 'E', ..., '71.81', 'NN', 'DO'],
       ['A', '276', 'F', ..., '143.62', 'NN', 'DO'],
       ['C', '414', 'E', ..., '71.81', 'NN', 'DO']],
      dtype='<U6')

In [22]:
categorical_features = [0,2,4,6,7,9,10,12,14,15,17,19,20]

In [23]:
categorical_names = {}
for feature in categorical_features:
    le = sklearn.preprocessing.LabelEncoder()
    le.fit(dataml[:, feature])
    dataml[:, feature] = le.transform(dataml[:, feature])
    categorical_names[feature] = le.classes_

In [24]:
dataml = dataml.astype(float)

In [25]:
encoder = sklearn.preprocessing.OneHotEncoder(categorical_features=categorical_features)

In [40]:
from sklearn.cross_validation import StratifiedKFold
eval_size = 0.10
kf = StratifiedKFold(labels,round(1. / eval_size))
train_indices , valid_indices = next(iter(kf))
x_train , y_train = dataml[train_indices] , labels[train_indices]
x_valid , y_valid = dataml[valid_indices] , labels[valid_indices]

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0])

In [26]:
np.random.seed(1)
train, test, labels_train, labels_test = sklearn.model_selection.train_test_split(dataml, labels, train_size=0.80)

C:\Users\Varma\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [27]:
encoder.fit(dataml)
encoded_train = encoder.transform(train)

In [45]:
encoded_train

<402x62 sparse matrix of type '<class 'numpy.float64'>'
	with 8442 stored elements in COOrdinate format>

In [28]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(encoded_train, labels_train)

In [29]:
clf.score(encoder.transform(test),labels_test)

0.67326732673267331

In [30]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

In [31]:
clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = GaussianNB()


In [32]:
eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard')

In [33]:
for clf, label in zip([clf1, clf2, clf3, eclf], ['Logistic Regression', 'Random Forest', 'naive Bayes', 'Ensemble']):
    scores = cross_val_score(clf, encoded_train.toarray(), labels_train, cv=5, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))    

Accuracy: 0.71 (+/- 0.01) [Logistic Regression]
Accuracy: 0.70 (+/- 0.04) [Random Forest]
Accuracy: 0.73 (+/- 0.02) [naive Bayes]
Accuracy: 0.71 (+/- 0.01) [Ensemble]


In [36]:
import xgboost


ModuleNotFoundError: No module named 'xgboost'

In [35]:
gbtree = xgboost.XGBClassifier(n_estimators=300, max_depth=5)


NameError: name 'xgboost' is not defined

In [ ]:
gbtree.fit(encoded_train, labels_train)

In [ ]:
sklearn.metrics.accuracy_score(labels_test, gbtree.predict(encoder.transform(test)))

In [ ]:
predict_fn = lambda x: gbtree.predict_proba(encoder.transform(x)).astype(float)

In [38]:
import pip

In [41]:
pip.main(['install', 'xgboost'])

  Using cached xgboost-0.7.post3.tar.gz


No files/directories in C:\Users\Varma\AppData\Local\Temp\pip-build-g1slgkvt\xgboost\pip-egg-info (from PKG-INFO)


1

In [ ]:
import lime
import lime.lime_tabular

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(train ,feature_names = feature_names,class_names=class_names,
                                                   categorical_features=categorical_features, 
                                                   categorical_names=categorical_names, kernel_width=3)